### Задание 9.5. Модуль ML-6 (HW-03)
Обучить модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

In [1]:
# Загрузка бибилиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# объявим функцию для обучения модели линейной регрессии
def func_LR (columns, data):
    """Функция - обучение модели линейной регрессии.

    Args:
        col (list): список признаков
        data (DataFrame): обработанный (подготовленный) дата-сет
    """

    y = data['price'] # выделяем целевой признак
    X = x[columns] # дата-сет с отобранными признаками

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40) # разделяем дата-сет на тренировочный и тестовый

    # обучаем модел линейной регрессии на тренирововчной выборке
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test) # делаем предсказание на тестовой выборке
    
    
    mae = mean_absolute_error(y_test, y_predicted)
    print('MAE: %.3f' % mae) # оценка качества модели Mean Absolute Error

In [3]:
# Загрузка дата-сета

#%%capture
#!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

data = pd.read_excel('data/data_ford_price.xlsx') # дата-сет загружен заранее

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


Произведем предобработку дата-сета. Исключим категориальные признаки, удалим пропущенные значения, выделим целевой признак в отдельный дата-сет.

In [5]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']] # исключаем категориальные признаки
data.dropna(inplace = True) # удаляем пропущенные значения

y = data['price'] # выделяем целевой признак
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40) # разделяем дата-сет на тренировочный и тестовый

In [6]:
x.info() # проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6837 entries, 0 to 7016
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       6837 non-null   int64  
 1   cylinders  6837 non-null   int64  
 2   odometer   6837 non-null   int64  
 3   lat        6837 non-null   float64
 4   long       6837 non-null   float64
 5   weather    6837 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 373.9 KB


1. Выделим 3 важных признака методом $RFE$

In [7]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out() # возвратим три наиболее важных признака
# 'year', 'cylinders', 'lat'

array(['year', 'cylinders', 'lat'], dtype=object)

In [8]:
col_RFE = ['year', 'cylinders', 'lat'] # создадим список наиболее важных признаков, полученных методом RFE

2. Выделим 3 важных признака методом фильтрации $SelectKBest$

In [9]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out() # возвратим три наиболее важных признака
# 'year', 'cylinders', 'odometer'

array(['year', 'cylinders', 'odometer'], dtype=object)

In [10]:
col_SKB = ['year', 'cylinders', 'odometer'] # создадим список наиболее важных признаков, полученных методом SelectKBest (SKB)

In [11]:
func_LR(col_RFE, data) # запускаем функцию обучения LR с признаками, полученными методом RFE

MAE: 5096.570


In [12]:
func_LR(col_SKB, data) # запускаем функцию обучения LR с признаками, полученными методом SKB

MAE: 4708.946


В проделанной работе произведен отбор трёх наиболее выжных признаков двумя методами (RFE и SelectKBest), произведено обучение модели линейной регресси на тренировочной выборке и предсказание обученной модели на тестовой выборке.<br>
Дата-сет предварительно прошел предобработку: удалены категориальные признаки и наблюдения, где встречаются нулевые значения.<br>
Метод RFE отобрал следующие три наиболее важных признака: year, cylinders, lat.<br>
Метод SelectKBest (SKB) отобрал три наиболее важных признака: year, cylinders, odometer.<br>
Результаты применения методов отличаются в третьем признаке.<br>
Оценка качества модели в двух случаях оценивалась методом Mean absolute error (MAE).<br>
Модель с отобранными признаками методом RFE имеет MAE 5096.57<br>
Модель с отобранными признаками методом SKB имеет MAE 4708.946<br>
Вторая модель показала более лучший результат, чем первая.<br>
Данный результат ожидаем, так как пробег действительно будет влиять на цену автомобиля больше, чем географическая широта места продажи автомобиля.